In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import re

In [3]:
import _thread
import threading

In [2]:
def filechecker(name):
    data=pd.read_csv(name)
    print('top 5 of',len(data),'rows from',name, 'data\n',data.head())

In [4]:
item_link=[]
def swg_crawler(link,filename='swg_brief.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    
    games=soup.find_all('div',{'class':'cell'})
    games
    for game in games:
        describe=game.find('div',{'class':'h6 name'}).text.strip()

        try:
            describe.split('\n')[0]
            name=describe.split('\n')[0]
        except:
            name=[]
            
        try:
            describe.split('\n')[1]
            discount=describe.split('\n')[1]
        except:
            discount=[]
            
        try:
            describe.split('\n')[2]
            tag=describe.split('\n')[2]
        except:
            tag=[]
            
        try:
            original_price=game.find('s',{'class':'text-muted'}).text.strip()
        except:
            original_price=[]
            
        try:
            discount_price=game.find('strong').text.strip()
        except:
            discount_price=[]
            
        try:
            deadline=game.find('div',{'class':'w-100'}).find('small').text.strip()
        except:
            deadline=[]
            
        
        link=game.find('a',{'class':'main-link'})['href']
        global item_link
        item_link.append('https://www.dekudeals.com'+link)
        
        with open(filename,"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow([name,discount,tag,original_price,discount_price,deadline])
        

In [5]:
def swg_item_crawler(link,filename='swg_detail.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    name=soup.find('span',{'class':'display-5'}).text
    #name
    info=soup.find_all('li',{'class':'list-group-item'})
    genre=[]
    publisher=[]
    size=[]
    release_date=[]
    metascore=[]
    userscore=[]
    language=[]
    play_mode=[]
    for i in info:
  
        if 'Genre' in i.text:
            genre= i.text.split(': ')[1:]
            #print(genre)
        if 'Publisher' in i.text:
            publisher=i.text.split(': ')[1:]
            #print(publisher)
        if 'Download' in i.text:
            size=i.text.split(': ')[1:]
            #print(size)
        if 'Release' in i.text:
            release_date=i.text.split(': ')[1:]
            #print(release_date)
        if 'Metacritic' in i.text:
            metascore=i.text.split(' ')[1]
            userscore=i.text.split(' ')[2]
            #print(metascore)
            #print(userscore)
        if 'Language' in i.text:
            language=i.text.split(': ')[1:]
            #print(language)
        if 'Play modes' in i.text:
            play_mode=i.text.split(': ')[1:]
            #print(play_mode)
    stores=soup.find_all('tr',{'class':'table-primary'})
    loc={}
    for store in stores:
        try:
            store_loc=store.find('img')['alt']
            version=store.find('td',{'class':'version'}).text.strip()
            price,discount=(store.find('div',{'class','btn-primary'}).text.strip().split('\n'))
            loc.update({store_loc:[version,price,discount]})
        except:
            pass
    #print(loc)
    with open(filename,"a+",encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow([name,genre,publisher,size,release_date,metascore,userscore,language,play_mode,loc])

In [6]:
with open('swg_brief.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Discount','Tag','Original_Price','Discount_Price','Deadline'])

with open('swg_detail.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Genre','Tag','Publisher','Download_Size','Release_Date','MetaScroe','UserScroe','Language','Play_Modes'])

In [7]:
#get the largest page number
link='https://www.dekudeals.com/hottest?page=1'
r=requests.get(link)
soup=BeautifulSoup(r.text,'html5lib')
maxpage=int(soup.find_all('a',{'class':'page-link'})[-2].text)+1
maxpage

17

In [8]:
main='https://www.dekudeals.com/'
link_list=[]
for i in range(1,maxpage):
    link_list.append(main+'hottest?page='+str(i))
link_list

['https://www.dekudeals.com/hottest?page=1',
 'https://www.dekudeals.com/hottest?page=2',
 'https://www.dekudeals.com/hottest?page=3',
 'https://www.dekudeals.com/hottest?page=4',
 'https://www.dekudeals.com/hottest?page=5',
 'https://www.dekudeals.com/hottest?page=6',
 'https://www.dekudeals.com/hottest?page=7',
 'https://www.dekudeals.com/hottest?page=8',
 'https://www.dekudeals.com/hottest?page=9',
 'https://www.dekudeals.com/hottest?page=10',
 'https://www.dekudeals.com/hottest?page=11',
 'https://www.dekudeals.com/hottest?page=12',
 'https://www.dekudeals.com/hottest?page=13',
 'https://www.dekudeals.com/hottest?page=14',
 'https://www.dekudeals.com/hottest?page=15',
 'https://www.dekudeals.com/hottest?page=16']

In [9]:
threads=[]
#build all the tasks 
for link in link_list:
    thread = threading.Thread(target=swg_crawler,args=(link,'swg_brief.csv'))
    threads.append(thread) 
 
                              
startTime=time.time()                              
#start all the tasks at once
for thread in threads:
    thread.start()
    
#wait for all the tasks to finish
for thread in threads:
    thread.join()
                              
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} pages'.format(len(link_list)))

      
filechecker('swg_brief.csv')


5.577842950820923 secs spend to craw these 16 pages
top 5 of 572 rows from swg_brief.csv data
                           Name Discount                   Tag Original_Price  \
0       Immortals Fenyx Rising     -17%  Matches previous low         $59.99   
1   Ginger: Beyond the Crystal     -75%     Lowest price ever         $19.99   
2   The Church in the Darkness     -75%  Matches previous low         $19.99   
3  Rad Rodgers Radical Edition     -75%  Matches previous low         $29.99   
4              Piczle Lines DX     -70%  Matches previous low         $14.99   

  Discount_Price               Deadline  
0         $49.94                     []  
1          $4.99   Sale ends November 1  
2          $4.99   Sale ends November 2  
3          $7.49  Sale ends in 23 hours  
4          $4.49   Sale ends November 1  


In [10]:
startTime=time.time()     
n=0
for link in item_link:
    n=n+1
    _thread.start_new_thread(swg_item_crawler, (link, ) )
    if n%5==0:
        n=0
        time.sleep(5)

                                  
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} games'.format(len(item_link)))

filechecker('swg_detail.csv')

Unhandled exception in thread started by <function swg_item_crawler at 0x7ff83af9a560>


ConnectionError: HTTPSConnectionPool(host='www.dekudeals.com', port=443): Max retries exceeded with url: /items/bloodstained-ritual-of-the-night (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff84003dc50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

Unhandled exception in thread started by <function swg_item_crawler at 0x7ff83af9a560>


ConnectionError: HTTPSConnectionPool(host='www.dekudeals.com', port=443): Max retries exceeded with url: /items/super-tennis-blast (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff83ed0e690>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

55.043842792510986 secs spend to craw these 572 games
top 5 of 548 rows from swg_detail.csv data
                            Name                                   Genre  \
0  Valfaris & Slain Double Pack  ['Platformer, Action, Action, Arcade']   
1                        Owlboy       ['Platformer, Adventure, Action']   
2                   Titan Quest                ['Action, Role-Playing']   
3                   Trüberbrook                   ['Adventure, Puzzle']   
4              Demon's Crystals    ['Action, Action, Arcade, Fighting']   

                 Tag      Publisher  \
0      ['Big Sugar']   ['1.157 GB']   
1   ['D-Pad Studio']     ['314 MB']   
2     ['THQ Nordic']  ['12.052 GB']   
3   ['Headup Games']   ['7.382 GB']   
4  ['Badland Games']   ['1.118 GB']   

                                       Download_Size Release_Date MetaScroe  \
0       ['NA', 'April 16, 2020EU', 'April 23, 2020']           []        []   
1                              ['February 13, 2018']      

In [11]:
startTime=time.time()     
for link in item_link:
    swg_item_crawler(link,'swg_detail_single.csv')
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} games'.format(len(item_link)))

filechecker('swg_detail_single.csv')

297.3889698982239 secs spend to craw these 572 games
top 5 of 571 rows from swg_detail_single.csv data
         Immortals Fenyx Rising                             ['Adventure']  \
0   Ginger: Beyond the Crystal         ['Platformer, Adventure, Action']   
1   The Church in the Darkness           ['Adventure, Action, Strategy']   
2  Rad Rodgers Radical Edition                    ['Platformer, Action']   
3              Piczle Lines DX  ['Puzzle, Training, Board Game, Action']   
4                  Trüberbrook                     ['Adventure, Puzzle']   

            ['Ubisoft'] ['13.164 GB']  \
0     ['Badland Games']   ['5.91 GB']   
1  ['Fellow Traveller']  ['1.447 GB']   
2        ['HandyGames']  ['4.264 GB']   
3        ['Rainy Frog']    ['281 MB']   
4      ['Headup Games']  ['7.382 GB']   

                               ['December  3, 2020']   [] [].1  \
0                              ['November 17, 2017']  tbd  tbd   
1  ['NA/EU', 'August  2, 2019JP', 'October  3, 20...   69  t